# Train Cats Dogs Dataset

In [2]:
import torch.nn as nn
from torchvision import models, transforms
import torch
from torchvision import datasets

In [3]:
class VGG16Sigmoid(nn.Module):
    def __init__(self):
        super(VGG16Sigmoid, self).__init__()
        self.model = models.vgg16(pretrained=False)
        self.model.classifier[6] = nn.Linear(4096, 1)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        x = self.model(x)
        return self.sigmoid(x)

In [ ]:
transform = transforms.Compose([
    transforms.Resize((250,250)),
    transforms.RandomHorizontalFlip(), 
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                            std=[0.229, 0.224, 0.225])
])

dataset = datasets.ImageFolder('cat-dogs-dataset/training_set/', transform=transform)
eval_dataset = datasets.ImageFolder('cat-dogs-dataset/test_set/', transform=transform)
target_to_class = {
    0: 'cat',
    1: 'dog'
}
dataloader = torch.utils.data.DataLoader(dataset, batch_size=32, shuffle=True)

train_size = int(0.9 * len(dataset))  
test_size = len(dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=2)
eval_loader = torch.utils.data.DataLoader(eval_dataset, batch_size=200, shuffle=True, num_workers=2)

In [5]:
print('Eval Data')

for images, labels in eval_loader:
    unique_labels, counts = labels.unique(return_counts=True)
    
    for label, count in zip(unique_labels, counts):
        print(f"{target_to_class[int(label.item())]}: {count.item()} images")

    break

print('\nTrain Data')

for images, labels in train_loader:
    unique_labels, counts = labels.unique(return_counts=True)
    
    for label, count in zip(unique_labels, counts):
        print(f"{target_to_class[int(label.item())]}: {count.item()} images")

    break

Eval Data


cat: 200 images


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x131ff7640>
Traceback (most recent call last):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1662, in __del__
    self._shutdown_workers()
  File "/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.10/site-packages/torch/utils/data/dataloader.py", line 1626, in _shutdown_workers
    w.join(timeout=_utils.MP_STATUS_CHECK_INTERVAL)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.10/multiprocessing/process.py", line 149, in join
    res = self._popen.wait(timeout)
  File "/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.10/multiprocessing/popen_fork.py", line 40, in wait
    if not wait([self.sentinel], timeout):
  File "/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.10/multiprocessing/connection.py", line 931, in wait
    ready = selector.select(timeout)
  File "/opt/homebrew/Caskroom/miniconda/b


Train Data
cat: 64 images


In [ ]:
device = 'mps' if torch.backends.mps.is_available() else 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
model = VGG16Sigmoid().to(device)

/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/homebrew/Caskroom/miniconda/base/envs/ML/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [ ]:
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

: 

In [ ]:
num_epochs = 20
loss_list = []
accuracy_list = []
for epoch in range(num_epochs):
    model.train()
    running_loss = correct = total = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        images, labels = images.to(device), labels.to(device)
        outputs = model(images).squeeze()
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        total += labels.size(0)
        
        predicted = outputs > 0.5.float()
        correct += (predicted==labels).sum().item()
        
    epoch_loss = running_loss / len(train_loader)   
    epoch_accuracy = (correct / total)*100
    
    loss_list.append(epoch_loss)
    accuracy_list.append(epoch_accuracy)
    
    scheduler.step()
    
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {epoch_loss:.4f} - Accuracy: {epoch_accuracy:.2f}%")

In [ ]:
torch.save(model, 'vgg16_sigmoid.pth')